In [1]:
import queue



In [2]:
q = queue.Queue()

for i in range(5):
    q.put(i)

while not q.empty():
    print(q.get(), end=' ')
print()

0 1 2 3 4 


In [3]:
q = queue.LifoQueue()

for i in range(5):
    q.put(i)

while not q.empty():
    print(q.get(), end=' ')
print()

4 3 2 1 0 


In [4]:
import functools
import queue
import threading


@functools.total_ordering
class Job:

    def __init__(self, priority, description):
        self.priority = priority
        self.description = description
        print('New job:', description)
        return

    def __eq__(self, other):
        try:
            return self.priority == other.priority
        except AttributeError:
            return NotImplemented

    def __lt__(self, other):
        try:
            return self.priority < other.priority
        except AttributeError:
            return NotImplemented


q = queue.PriorityQueue()

q.put(Job(3, 'Mid-level job'))
q.put(Job(10, 'Low-level job'))
q.put(Job(1, 'Important job'))


def process_job(q):
    while True:
        next_job = q.get()
        print('Processing job:', next_job.description)
        q.task_done()


workers = [
    threading.Thread(target=process_job, args=(q,)),
    threading.Thread(target=process_job, args=(q,)),
]
for w in workers:
    w.setDaemon(True)
    w.start()

q.join()

New job: Mid-level job
New job: Low-level job
New job: Important job
Processing job: Important job
Processing job: Mid-level job
Processing job: Low-level job


In [2]:
from queue import Queue
import threading
import time
import urllib
from urllib.parse import urlparse

import feedparser

# Set up some global variables
num_fetch_threads = 2
enclosure_queue = Queue()

# A real app wouldn't use hard-coded data...
feed_urls = [
    'http://talkpython.fm/episodes/rss',
]


def message(s):
    print('{}: {}'.format(threading.current_thread().name, s))

In [3]:
def download_enclosures(q):
    """This is the worker thread function.
    It processes items in the queue one after
    another.  These daemon threads go into an
    infinite loop, and exit only when
    the main thread ends.
    """
    while True:
        message('looking for the next enclosure')
        url = q.get()
        filename = url.rpartition('/')[-1]
        message('downloading {}'.format(filename))
        response = urllib.request.urlopen(url)
        data = response.read()
        # Save the downloaded file to the current directory
        message('writing to {}'.format(filename))
        with open(filename, 'wb') as outfile:
            outfile.write(data)
        q.task_done()

In [4]:
for i in range(num_fetch_threads):
    worker = threading.Thread(
        target=download_enclosures,
        args=(enclosure_queue,),
        name='worker-{}'.format(i),
    )
    worker.setDaemon(True)
    worker.start()

worker-0: looking for the next enclosure
worker-1: looking for the next enclosure


In [5]:
# Download the feed(s) and put the enclosure URLs into
# the queue.
for url in feed_urls:
    response = feedparser.parse(url, agent='fetch_podcasts.py')
    for entry in response['entries'][:5]:
        for enclosure in entry.get('enclosures', []):
            parsed_url = urlparse(enclosure['url'])
            message('queuing {}'.format(
                parsed_url.path.rpartition('/')[-1]))
            enclosure_queue.put(enclosure['url'])

MainThread: queuing python-on-windows-is-ok-actually.mp3
MainThread: queuing your-education-will-be-live-streamed.mp3
MainThread: queuing opal-full-stack-health-care-apps.mp3
MainThread: queuing a-guided-tour-of-the-cpython-source-code.mp3
MainThread: queuing bayesian-foundations.mp3
worker-0: downloading python-on-windows-is-ok-actually.mp3
worker-1: downloading your-education-will-be-live-streamed.mp3


In [6]:
# Now wait for the queue to be empty, indicating that we have
# processed all of the downloads.
message('*** main thread waiting')
enclosure_queue.join()
message('*** done')

MainThread: *** main thread waiting
worker-1: writing to your-education-will-be-live-streamed.mp3
worker-1: looking for the next enclosure
worker-1: downloading opal-full-stack-health-care-apps.mp3
worker-0: writing to python-on-windows-is-ok-actually.mp3
worker-0: looking for the next enclosure
worker-0: downloading a-guided-tour-of-the-cpython-source-code.mp3
worker-1: writing to opal-full-stack-health-care-apps.mp3
worker-1: looking for the next enclosure
worker-1: downloading bayesian-foundations.mp3
worker-0: writing to a-guided-tour-of-the-cpython-source-code.mp3
worker-0: looking for the next enclosure
worker-1: writing to bayesian-foundations.mp3
worker-1: looking for the next enclosureMainThread: *** done

